In [ ]:
import torch
import os
from torch.utils.data import TensorDataset,ConcatDataset,DataLoader
import matplotlib.pyplot as plt
from torch import nn
from torch.nn.functional import relu
from tqdm import tqdm
from sklearn.preprocessing import RobustScaler
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from sklearn.metrics import f1_score
import numpy as np

import torch
from torch import nn

from lib.utils import get_dataloader
from lib.utils import SleepStageClassifier
from lib.utils import ekyn_ids,snezana_mice_ids,courtney_ids
from sklearn.metrics import ConfusionMatrixDisplay,classification_report
from lib.utils import calculate_f1,plot_training_progress
import torch
eeg,emg,y = torch.load(f'../pt_courtney_eeg_emg/22-Oct-H.pt')

mean = eeg.flatten().mean()
std = eeg.flatten().std()
eeg = (eeg - mean) / (std + 1e-5)
eeg = eeg[:,::10]

mean = emg.flatten().mean()
std = emg.flatten().std()
emg = (emg - mean) / (std + 1e-5)
emg = emg[:,::10]

X = torch.cat([eeg.unsqueeze(1),emg.unsqueeze(1)],dim=1)

class SleepStageClassifier(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.stem = nn.Conv1d(in_channels=2,out_channels=64,kernel_size=7)
        self.mp1 = nn.MaxPool1d(kernel_size=2)
        self.convs = nn.ModuleList([nn.Conv1d(in_channels=64,out_channels=64,kernel_size=3) for _ in range(1)])
        self.gap = nn.AdaptiveAvgPool1d(output_size=1)
        self.fc1 = nn.Linear(in_features=64,out_features=32)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(in_features=32,out_features=3)
    def forward(self,x):
        x = self.stem(x)
        x = nn.functional.relu(x)
        for conv in self.convs:
            x = conv(x)
            x = nn.functional.relu(x)
        x = self.gap(x)
        x = x.flatten(1,2)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
device = 'cuda'
conditions = ['PF','Vehicle']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SleepStageClassifier()
optimizer = torch.optim.AdamW(model.parameters(),lr=3e-4)
criterion = nn.CrossEntropyLoss()

/tmp/ipykernel_356628/989082379.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eeg,emg,y = torch.load(f'../pt_courtney_eeg_emg/22-Oct-H.pt')


In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
trainloader = DataLoader(TensorDataset(X_train,y_train),batch_size=512,shuffle=True)
testloader = DataLoader(TensorDataset(X_test,y_test),batch_size=512,shuffle=True)

In [ ]:

validation_frequency_epochs = 1
best_dev_loss = torch.inf
best_dev_loss_epoch = 0
best_dev_f1 = 0
best_dev_f1_epoch = 0
ma_window_size = 10

model.to(device)
model.train()

trainlossi = []
testlossi = []
train_f1s = []
test_f1s = []

for epoch in range(1000):
    epoch_train_f1s = []  # Collect F1s for each batch in this epoch
    epoch_train_losses = []  # Collect losses for each batch in this epoch
    
    for Xi, yi in tqdm(trainloader):
        Xi, yi = Xi.to(device), yi.to(device)    
        logits = model(Xi)
        optimizer.zero_grad()
        loss = criterion(logits, yi)
        loss.backward()
        optimizer.step()
        
        # Calculate and store F1 for this batch
        batch_f1 = calculate_f1(logits, yi)
        epoch_train_f1s.append(batch_f1)
        epoch_train_losses.append(loss.item())
    
    # Add average loss and F1 for this epoch
    trainlossi.extend(epoch_train_losses)
    train_f1s.append(np.mean(epoch_train_f1s))
    
    if epoch % validation_frequency_epochs == 0:
        model.eval()
        all_test_preds = []
        all_test_labels = []
        test_losses = []

        with torch.no_grad():
            for Xi, yi in testloader:
                Xi, yi = Xi.to(device), yi.to(device)
                logits = model(Xi)
                loss = criterion(logits, yi)
                test_losses.append(loss.item())
                all_test_preds.append(logits.argmax(dim=1))
                all_test_labels.append(yi.argmax(dim=1))
            all_test_preds = torch.cat(all_test_preds).cpu()
            all_test_labels = torch.cat(all_test_labels).cpu()
            avg_test_loss = np.mean(test_losses)
            test_f1 = f1_score(all_test_labels, all_test_preds, average='macro')

            testlossi.append(avg_test_loss)
            test_f1s.append(test_f1)

            # Track best model by loss
            if avg_test_loss < best_dev_loss:
                best_dev_loss = avg_test_loss
                best_dev_loss_epoch = epoch
                torch.save(model.state_dict(), '../models/best_model_by_loss.pt')
                
            # Track best model by F1
            if test_f1 > best_dev_f1:
                best_dev_f1 = test_f1
                best_dev_f1_epoch = epoch
                torch.save(model.state_dict(), '../models/best_model_by_f1.pt')
                
            # Call the updated plotting function with both loss and F1 data
            plot_training_progress(
                trainlossi,
                testlossi,
                train_f1s,
                test_f1s,
                ma_window_size,
                '../models/training_metrics.jpg'
            )
            print(f"Epoch {epoch}: Train Loss: {np.mean(epoch_train_losses):.4f}, Test Loss: {avg_test_loss:.4f}")
            print(f"Train F1: {train_f1s[-1]:.4f}, Test F1: {test_f1:.4f}")
            print(f"Best Test Loss: {best_dev_loss:.4f} (Epoch {best_dev_loss_epoch})")
            print(f"Best Test F1: {best_dev_f1:.4f} (Epoch {best_dev_f1_epoch})")


y = torch.vstack([torch.vstack([model(Xi.to(device)).softmax(dim=1).argmax(dim=1).detach().cpu(),yi.argmax(dim=1).detach().cpu()]).T for Xi,yi in trainloader])
y_pred = y[:,0]
y_true = y[:,1]
print(classification_report(y_true=y_true,y_pred=y_pred))
ConfusionMatrixDisplay.from_predictions(y_true,y_pred)

y = torch.vstack([torch.vstack([model(Xi.to(device)).softmax(dim=1).argmax(dim=1).detach().cpu(),yi.argmax(dim=1).detach().cpu()]).T for Xi,yi in testloader])
y_pred = y[:,0]
y_true = y[:,1]
print(classification_report(y_true=y_true,y_pred=y_pred))
ConfusionMatrixDisplay.from_predictions(y_true,y_pred)

100%|██████████| 65/65 [00:00<00:00, 70.27it/s]


Epoch 0: Train Loss: 0.2215, Test Loss: 0.1731
Train F1: 0.3464, Test F1: 0.4891
Best Test Loss: 0.1731 (Epoch 0)
Best Test F1: 0.4891 (Epoch 0)


100%|██████████| 65/65 [00:00<00:00, 102.48it/s]


Epoch 1: Train Loss: 0.1432, Test Loss: 0.1074
Train F1: 0.5444, Test F1: 0.5626
Best Test Loss: 0.1074 (Epoch 1)
Best Test F1: 0.5626 (Epoch 1)


100%|██████████| 65/65 [00:00<00:00, 84.97it/s]


...

100%|██████████| 65/65 [00:00<00:00, 102.53it/s]


Epoch 199: Train Loss: 0.0495, Test Loss: 0.0513
Train F1: 0.8882, Test F1: 0.8839
Best Test Loss: 0.0510 (Epoch 195)
Best Test F1: 0.8851 (Epoch 160)


100%|██████████| 65/65 [00:00<00:00, 102.55it/s]


Epoch 200: Train Loss: 0.0493, Test Loss: 0.0539
Train F1: 0.8896, Test F1: 0.8832
Best Test Loss: 0.0510 (Epoch 195)
Best Test F1: 0.8851 (Epoch 160)


100%|██████████| 65/65 [00:00<00:00, 102.55it/s]
